In [11]:
import requests
from bs4 import BeautifulSoup
import json
import csv
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

# 1. Web Scraping
url = "https://report-tv.al/"  # Target website
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

data = []

# Improved location extraction function
def extract_location_from_title(title):
    cities = ["Tiranë", "Durrës", "Fier", "Korçë", "Elbasan", "Gjirokastër", "Has", "Shkodër", "Berat"]
    normalized_title = title.lower()
    for city in cities:
        if city.lower() in normalized_title:
            return city
    return "Location not found"

# OpenWeather API details
api_url = "https://api.openweathermap.org/data/2.5/weather"
api_key = "955543ddcb6f10847084aab472d9ed80"  # Replace with your actual API key

# Extract titles, locations, and weather
for article in soup.find_all('h2'):  # Adjust tag or class as needed
    title = article.get_text(strip=True)
    location = extract_location_from_title(title)
    
    weather_info = "Invalid location"
    if location != "Location not found":
        params = {
            "q": location,
            "appid": api_key,
            "units": "metric"  # Get temperature in Celsius
        }
        weather_response = requests.get(api_url, params=params).json()
        if "main" in weather_response:
            weather_info = {
                "temperature": weather_response["main"]["temp"],
                "description": weather_response["weather"][0]["description"]
            }
        else:
            weather_info = "Weather data not found"
    
    data.append({"title": title, "location": location, "weather": weather_info})

# Save to JSON
json_filename = "report_tv_titles_with_weather.json"
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

# Save to CSV
csv_filename = "report_tv_titles_with_weather.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["title", "location", "weather"])
    writer.writeheader()
    writer.writerows(data)

# 2. Encryption - AES
key = os.urandom(32)  # Generate a random 256-bit key
iv = os.urandom(16)   # Generate a random IV

cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
encryptor = cipher.encryptor()

with open(json_filename, 'rb') as f:
    plaintext = f.read()

ciphertext = encryptor.update(plaintext) + encryptor.finalize()

encrypted_filename = "data_encrypted.aes"
with open(encrypted_filename, 'wb') as f:
    f.write(iv + ciphertext)  # Prepend IV for decryption

# RSA Encryption for Key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

encrypted_key = public_key.encrypt(
    key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

key_filename = "encrypted_key.pem"
with open(key_filename, 'wb') as f:
    f.write(encrypted_key)

print(f"Data scraped, encrypted, and saved to {json_filename}, {csv_filename}, and {encrypted_filename}.")


Data scraped, encrypted, and saved to report_tv_titles_with_weather.json, report_tv_titles_with_weather.csv, and data_encrypted.aes.
